In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mysql.connector
import json

In [2]:
#server = '192.168.10.138' remote
server = '127.0.0.1'

conn = mysql.connector.connect(user='root', password='gmw6504192658',
                                            host=server, database='nd', charset='utf8',
                                            use_unicode=True)
cursor = conn.cursor()  # Database operation

In [3]:
df_allndlist= pd.read_sql('SELECT * FROM nd', con=conn)

In [4]:
df_allndlist.head(2)

,id,symbol,name,sector,industry,country,board,prof_margin,ppe,vol,descShort,url_reuter,url_marketwatch,ipo_year,NIGR_result
0,1,PIH,"1347 Property Insurance Holdings, Inc.",Financial,Property & Casualty Insurance,USA,[NASD],42.00%,23.09,"4,501","1347 Property Insurance Holdings, Inc., throug...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/PI...,2014.0,"{""2014-2013"": ""-"", ""2015-2014"": ""-153.18%"", ""2..."
1,2,VNET,"21Vianet Group, Inc.",Technology,Information Technology Services,China,[NASD],-16.70%,-,"216,347","21Vianet Group, Inc. provides carrier-neutral ...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/VN...,2011.0,"{""2014-2013"": ""-619.18%"", ""2015-2014"": ""-20.55..."


In [5]:
def CustomParser(data):
    j1 = json.loads(data)
    return j1

In [6]:
df_allndlist['NIGR_result'] = df_allndlist['NIGR_result'].apply(CustomParser)
df_allndlist[sorted(df_allndlist['NIGR_result'][0].keys())] = df_allndlist['NIGR_result'].apply(pd.Series)
df_allndlist.drop(columns='NIGR_result',inplace = True)

In [7]:
df_allndlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 18 columns):
id                 890 non-null int64
symbol             890 non-null object
name               890 non-null object
sector             890 non-null object
industry           890 non-null object
country            890 non-null object
board              890 non-null object
prof_margin        890 non-null object
ppe                657 non-null object
vol                890 non-null object
descShort          890 non-null object
url_reuter         890 non-null object
url_marketwatch    890 non-null object
ipo_year           890 non-null object
2014-2013          890 non-null object
2015-2014          890 non-null object
2016-2015          890 non-null object
2017-2016          890 non-null object
dtypes: int64(1), object(17)
memory usage: 125.2+ KB


In [8]:
df_allndlist['vol'] = df_allndlist['vol'].apply(lambda x: float(x.replace(',','')) if x!= '0.00%' else 0)
df_allndlist['ppe'] = df_allndlist['ppe'].apply(lambda x: float(x) if (x!= '-' and x is not None)  else 0)
df_allndlist['ipo_year'] = df_allndlist['ipo_year'].apply(lambda x: int(x[:-2]))
df_allndlist['prof_margin'] = df_allndlist['prof_margin'].apply(lambda x: round(float((x[:-1]))/100,4))    

In [9]:
df_allndlist.head(2)

,id,symbol,name,sector,industry,country,board,prof_margin,ppe,vol,descShort,url_reuter,url_marketwatch,ipo_year,2014-2013,2015-2014,2016-2015,2017-2016
0,1,PIH,"1347 Property Insurance Holdings, Inc.",Financial,Property & Casualty Insurance,USA,[NASD],0.420,23.09,4501.0,"1347 Property Insurance Holdings, Inc., throug...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/PI...,2014,-,-153.18%,100.66%,"2,572.73%"
1,2,VNET,"21Vianet Group, Inc.",Technology,Information Technology Services,China,[NASD],-0.167,0.00,216347.0,"21Vianet Group, Inc. provides carrier-neutral ...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/VN...,2011,-619.18%,-20.55%,-39.80%,-42.19%


In [10]:
def p2f(x):
    if x!= '-':
        return float((x.replace(',', "").strip('%')))/100
    else:
        return 0
#p2f('24816.87%')
df_allndlist['2017-2016'] = df_allndlist['2017-2016'].apply(p2f)
df_allndlist['2016-2015'] = df_allndlist['2016-2015'].apply(p2f)
df_allndlist['2015-2014'] = df_allndlist['2015-2014'].apply(p2f)
df_allndlist['2014-2013'] = df_allndlist['2014-2013'].apply(p2f)

In [11]:
df_allndlist.head(2)

,id,symbol,name,sector,industry,country,board,prof_margin,ppe,vol,descShort,url_reuter,url_marketwatch,ipo_year,2014-2013,2015-2014,2016-2015,2017-2016
0,1,PIH,"1347 Property Insurance Holdings, Inc.",Financial,Property & Casualty Insurance,USA,[NASD],0.420,23.09,4501.0,"1347 Property Insurance Holdings, Inc., throug...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/PI...,2014,0.0000,-1.5318,1.0066,25.7273
1,2,VNET,"21Vianet Group, Inc.",Technology,Information Technology Services,China,[NASD],-0.167,0.00,216347.0,"21Vianet Group, Inc. provides carrier-neutral ...",http://www.reuters.com/finance/stocks/financia...,https://www.marketwatch.com/investing/stock/VN...,2011,-6.1918,-0.2055,-0.3980,-0.4219


In [12]:
#df_allnd100list.rename(index=str, columns={"shares_out": "shares_out(million)", "mark_cap": "mark_cap(billion)"},inplace= True)

In [13]:
#df_allnd100list.ix[df_allnd100list['symbol']=='AAPL']

In [14]:
#df_allnd100list.ix[df_allnd100list['shares_out'].idxmax()]

In [15]:
df_allndlist.to_csv('./csv/list_nd_all.csv',encoding='utf-8',index=False)